In [234]:
import numpy as np
import pandas as pd
import json
import tweepy
import seaborn as sns
import os
import requests as req
import calendar
import pytz
import time
import got3

from datetime import datetime, date, timedelta

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()



In [235]:
api_dir = os.path.dirname(os.path.dirname(os.path.realpath('keys')))
file_name = os.path.join(api_dir + "//keys", "api_keys.json")
data = json.load(open(file_name))

gkey = data['google_api_key']
consumer_key = data['twitter_consumer_key']
consumer_secret = data['twitter_consumer_secret']
access_token = data['twitter_access_token']
access_token_secret = data['twitter_access_token_secret']


In [236]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())



In [237]:
#coin list by symbol in descending order of market cap
#bitcoin - ethereum - ripple - bitcoin cash - cardano - litecoin - stella - neo - eos - xem  
coin_list = ['BTC', 'ETH', 'XRP', 'BCH', 'ADA', 'LTC', 'XLM', 'NEO', 'EOS', 'XEM']



In [238]:
#URLs
social_stats_url = "https://www.cryptocompare.com/api/data/socialstats/?id="

coin_list_url = "https://www.cryptocompare.com/api/data/coinlist/"

coin_historical_url = "https://min-api.cryptocompare.com/data/pricehistorical?fsym="
coin_historical_url_second = "&tsyms=USD&ts="



In [239]:
#variables 
coin_list_response = []

coin_data_df = pd.DataFrame(columns=["symbol", "date", "coin_name", "coin_price", 
                                     "compound_average",
                                     "positive_average", "neutral_average", "negative_average"
                                   ])



In [240]:
#date
year = 2018
month = 1
day = 1

#sampling max number of tweets per day per coin
max_tweets = 25

In [241]:
def GetCoinList():
    response = req.get(coin_list_url).json()

    count = 0
    for each in response["Data"]:
        count = count + 1
    
    print(count)
    return response


def GetCoinId(symbol):
    for each in coin_list_response["Data"]:
        row = coin_list_response["Data"][symbol]
        if row["Name"] == symbol:
            return int(row["Id"])


def GetCoinName(symbol):
    for each in coin_list_response["Data"]:
        row = coin_list_response["Data"][symbol]
        if row["Name"] == symbol:
            return row["CoinName"].lower()

               
def GetHistoricalPrice(symbol, utctime):
    #url to get historical price
    url = coin_historical_url + symbol + coin_historical_url_second + str(utctime)
    #get the historical price
    price = req.get(url).json()
    return float(price[symbol]["USD"])


def GetSocialStats(coin_id):
    url = social_stats_url + coin_id 
    return req.get(url).json()


def GetUtcTime(year,month,day):
    utc = calendar.timegm(datetime(year,month,day).utctimetuple())
    return utc

    
def AnalyzeTweets(coin_name, until_date):
   
    tweetCriteria = got3.manager.TweetCriteria().setUntil(until_date).setQuerySearch(coin_name).setMaxTweets(max_tweets)

    for i in range(max_tweets):
        #each tweet
        tweet = got3.manager.TweetManager.getTweets(tweetCriteria)[i]      
        #run Vader analysis
        compound = analyzer.polarity_scores(tweet.text)["compound"]
        pos = analyzer.polarity_scores(tweet.text)["pos"]
        neu = analyzer.polarity_scores(tweet.text)["neu"]
        neg = analyzer.polarity_scores(tweet.text)["neg"]
        #add each score to an appropriate array
        compound_list.append(compound)
        positive_list.append(pos)
        negative_list.append(neg)
        neutral_list.append(neu)





In [242]:
#get coin list
coin_list_response = GetCoinList()

count = 0

#build coin data
while count < 2:
    #get UTC time, one day at a time
    utc_time = GetUtcTime(year,month,day)
    until_date = date(year,month,day).strftime("%Y-%m-%d")  
    print(until_date)
 
    #process one coin at a time
    for symbol in coin_list:
        coin_name = GetCoinName(symbol)
        print(symbol,coin_name)
        
        #calculate Vader scores
        compound_list = []
        positive_list = []
        negative_list = []
        neutral_list = []
        AnalyzeTweets(coin_name,until_date)
        
        print(np.mean(compound_list))
        
        row = {"symbol": symbol, 
               "date": until_date,
               "coin_name": coin_name,
               "coin_price": GetHistoricalPrice(symbol,utc_time),
               "compound_average": float(np.mean(compound_list)),
               "positive_average": float(np.mean(positive_list)), 
               "neutral_average": float(np.mean(neutral_list)), 
               "negative_average": float(np.mean(negative_list)) }
        
        coin_data_df = coin_data_df.append(row, ignore_index=True)   
        
    count = count + 1
    
    #get the next date
    next_date = date(year,month,day) + timedelta(1)
    year = next_date.year
    month = next_date.month
    day = next_date.day
    
coin_data_df

2112
2018-01-01
BTC bitcoin
0.223233333333
ETH ethereum
0.481333333333
XRP ripple
0.480466666667
BCH bitcoin cash / bcc
0.1576
ADA cardano
0.525733333333
LTC litecoin
0.0614333333333
XLM stellar
0.0566
NEO neo
-0.0754333333333
EOS eos
0.1468
XEM nem
0.0
2018-01-02
BTC bitcoin
0.2423
ETH ethereum
0.0
XRP ripple
0.1589
BCH bitcoin cash / bcc
0.0
ADA cardano
0.383533333333
LTC litecoin
0.0674333333333
XLM stellar
-0.180766666667
NEO neo
0.1405
EOS eos
0.0164
XEM nem
0.0


,symbol,date,coin_name,coin_price,compound_average,positive_average,neutral_average,negative_average
0,BTC,2018-01-01,bitcoin,13444.8800,0.223233,0.042333,0.957667,0.000000
1,ETH,2018-01-01,ethereum,756.2000,0.481333,0.120333,0.879667,0.000000
2,XRP,2018-01-01,ripple,2.0500,0.480467,0.084667,0.915333,0.000000
3,BCH,2018-01-01,bitcoin cash / bcc,2319.1200,0.157600,0.102000,0.865667,0.032333
4,ADA,2018-01-01,cardano,0.7022,0.525733,0.109333,0.879333,0.011000
5,LTC,2018-01-01,litecoin,224.3400,0.061433,0.083000,0.896000,0.020667
6,XLM,2018-01-01,stellar,0.4840,0.056600,0.135667,0.803667,0.061000
7,NEO,2018-01-01,neo,78.0900,-0.075433,0.048667,0.890667,0.060667
8,EOS,2018-01-01,eos,7.6400,0.146800,0.037333,0.962667,0.000000
9,XEM,2018-01-01,nem,1.0100,0.000000,0.000000,1.000000,0.000000


In [243]:
file_name = os.path.join("resources", "coin_tweet_analysis_7_days_20180101.csv")
coin_data_df.to_csv(file_name)


In [244]:
coin_data_df = coin_data_df.reset_index()
coin_data_df

,index,symbol,date,coin_name,coin_price,compound_average,positive_average,neutral_average,negative_average
0,0,BTC,2018-01-01,bitcoin,13444.8800,0.223233,0.042333,0.957667,0.000000
1,1,ETH,2018-01-01,ethereum,756.2000,0.481333,0.120333,0.879667,0.000000
2,2,XRP,2018-01-01,ripple,2.0500,0.480467,0.084667,0.915333,0.000000
3,3,BCH,2018-01-01,bitcoin cash / bcc,2319.1200,0.157600,0.102000,0.865667,0.032333
4,4,ADA,2018-01-01,cardano,0.7022,0.525733,0.109333,0.879333,0.011000
5,5,LTC,2018-01-01,litecoin,224.3400,0.061433,0.083000,0.896000,0.020667
6,6,XLM,2018-01-01,stellar,0.4840,0.056600,0.135667,0.803667,0.061000
7,7,NEO,2018-01-01,neo,78.0900,-0.075433,0.048667,0.890667,0.060667
8,8,EOS,2018-01-01,eos,7.6400,0.146800,0.037333,0.962667,0.000000
9,9,XEM,2018-01-01,nem,1.0100,0.000000,0.000000,1.000000,0.000000
